In [5]:
import numpy as np
import pandas as pd
import sys
from pathlib import Path
sys.path.append('..') 
import platform
import os
from pathlib import Path
import shutil
from utils.basics.data_org import curr_computer, move_subfolders
from pathlib import Path
import shutil
from utils.behavior.session_utils import load_session_df, parse_session_string
from utils.behavior.lick_analysis import clean_up_licks, parse_lick_trains
from scipy.io import loadmat
from itertools import chain
from matplotlib import pyplot as plt
from IPython.display import display
from scipy.signal import find_peaks
from harp.clock import align_timestamps_to_anchor_points
import json
from sklearn.linear_model import LinearRegression
# %matplotlib inline
%matplotlib widget

In [ ]:
session_df, licks_L, licks_R = load_session_df('m699461d20231222')

In [54]:
licks_L, licks_R, fig = clean_up_licks(licks_L, licks_R, plot=False)
parsed_licks_L, _ = parse_lick_trains(licks_L)

In [8]:
session = 'm699461d20231222'
label = 'CombinewithKH'
def load_session_FP(session, label, plot=False):
    session_df, licks_L, _ = load_session_df(session)
    session_path = parse_session_string(session)
    file_name = f'{session}_photometry{label}.mat'
    photometry_file = os.path.join(session_path['sortedFolder'], file_name)
    photometry_json = os.path.join(session_path['photometryPath'], f'{session}.json')
    signal_mat = loadmat(photometry_file)
    with open(photometry_json, "r") as file:
        location_info = json.load(file)
    # Get all files in the directory
    allFiles = os.listdir(fpDir)

    # Filter only files (ignore directories)
    allFiles = [f for f in allFiles if not os.path.isdir(os.path.join(fpDir, f))]
    time_stamps = None
    signal_region_raw = {}
    # Load signal files
    for channel in channels: 
        curr_sig = {}
        channelInd = [f for f in allFiles if f'FIP_Data{channel}' in f]
        if len(channelInd) > 0:
            channelSigData = pd.read_csv(os.path.join(fpDir, channelInd[0]), header=None).to_numpy()
            if channel == 'G':
                time_stamps = channelSigData[:, 0]
            # curr_sig[channel] = channelSigData[:, 1:1 + len(location_info)]
            for key, value in location_info.items():
                print(f"Channel {channel}:Region {value} recorded at fiber {key}")
                curr_sig[value] = np.array(channelSigData[:, int(key)+1])
        signal_region_raw[channel] = curr_sig

    for color in channels:
        curr_sig = signal_region_raw[color]
        for key, value in location_info.items():
            curr_sig[value] = curr_sig[value][(time_stamps <= signal_region['time'][-1]) & (time_stamps >= signal_region['time'][0])]


    signal_region_raw['time'] = time_stamps
    signal_region_raw['time_in_beh'] = signal_region['time_in_beh']

    if plot:
        fig, ax = plt.subplots()
        ax.plot(signal_region['time_in_beh'], signal_region[location_info['0']], label='channel 0')
        ax2 = ax.twinx()
        ax2.hist(licks_L, bins=100, alpha=0.5, label='Licks L')
        ax.set_title('Alignment Check')
        plt.show()
        return signal_region, fig
    else:
        return signal_region


In [80]:
session = 'm699461d20231222'
label = 'CombinewithKH'
signal_region = load_session_df(session, label, plot=False)

TypeError: load_session_df() missing 1 required positional argument: 'label'